In [35]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import matplotlib as mpl
import json
import pickle
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

# for VS code plotting
# mpl.rcParams.update(_VSCode_defaultMatplotlib_Params)
#sns.set_style(rc = {'figure.facecolor':'white'})

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
cf.go_offline()
import plotly.express as px

In [2]:
from googletrans import Translator

In [3]:
# read in data for india
df = pd.read_csv('data/INvideos.csv', parse_dates=['trending_date', 'publish_time'])
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,kzwfHumJyYc,17.14.11,Sharry Mann: Cute Munda ( Song Teaser) | Parmi...,Lokdhun Punjabi,1,2017-11-12 12:20:39+00:00,"sharry mann|""sharry mann new song""|""sharry man...",1096327,33966,798,882,https://i.ytimg.com/vi/kzwfHumJyYc/default.jpg,False,False,False,Presenting Sharry Mann latest Punjabi Song Cu...
1,zUZ1z7FwLc8,17.14.11,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं...",HJ NEWS,25,2017-11-13 05:43:56+00:00,"पीरियड्स के समय|""पेट पर पति करता ऐसा""|""देखकर द...",590101,735,904,0,https://i.ytimg.com/vi/zUZ1z7FwLc8/default.jpg,True,False,False,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं..."
2,10L1hZ9qa58,17.14.11,Stylish Star Allu Arjun @ ChaySam Wedding Rece...,TFPC,24,2017-11-12 15:48:08+00:00,Stylish Star Allu Arjun @ ChaySam Wedding Rece...,473988,2011,243,149,https://i.ytimg.com/vi/10L1hZ9qa58/default.jpg,False,False,False,Watch Stylish Star Allu Arjun @ ChaySam Weddin...
3,N1vE8iiEg64,17.14.11,Eruma Saani | Tamil vs English,Eruma Saani,23,2017-11-12 07:08:48+00:00,"Eruma Saani|""Tamil Comedy Videos""|""Films""|""Mov...",1242680,70353,1624,2684,https://i.ytimg.com/vi/N1vE8iiEg64/default.jpg,False,False,False,This video showcases the difference between pe...
4,kJzGH0PVQHQ,17.14.11,why Samantha became EMOTIONAL @ Samantha naga ...,Filmylooks,24,2017-11-13 01:14:16+00:00,"Filmylooks|""latest news""|""telugu movies""|""telu...",464015,492,293,66,https://i.ytimg.com/vi/kJzGH0PVQHQ/default.jpg,False,False,False,why Samantha became EMOTIONAL @ Samantha naga ...


In [4]:
# parse trending date
df['trending_date'] = pd.to_datetime(df['trending_date'], format='%y.%d.%m')
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37352 entries, 0 to 37351
Data columns (total 16 columns):
video_id                  37352 non-null object
trending_date             37352 non-null datetime64[ns]
title                     37352 non-null object
channel_title             37352 non-null object
category_id               37352 non-null int64
publish_time              37352 non-null datetime64[ns, UTC]
tags                      37352 non-null object
views                     37352 non-null int64
likes                     37352 non-null int64
dislikes                  37352 non-null int64
comment_count             37352 non-null int64
thumbnail_link            37352 non-null object
comments_disabled         37352 non-null bool
ratings_disabled          37352 non-null bool
video_error_or_removed    37352 non-null bool
description               36791 non-null object
dtypes: bool(3), datetime64[ns, UTC](1), datetime64[ns](1), int64(5), object(6)
memory usage: 3.8+ MB


In [5]:
# drop duplicate rows
print(f'shape before dropping duplicates - {df.shape}')
df = df.drop_duplicates()
print(f'shape after dropping duplicates - {df.shape}')

shape before dropping duplicates - (37352, 16)
shape after dropping duplicates - (33089, 16)


In [6]:
# read in category mapping json
with open('data/IN_category_id.json') as file:
    cat_map = json.load(file) 

In [7]:
# preprocess category mapping to pull in category id vs title
df_cat_map = pd.DataFrame()
num_categories = df.category_id.unique().shape[0]
df_cat_map['category_id'] = ['']*num_categories
df_cat_map['category_title'] = ['']*num_categories
for i,x in enumerate(cat_map['items']):
    df_cat_map.loc[i, 'category_id'] = x['id']
    df_cat_map.loc[i, 'category_title'] = x['snippet']['title']
df_cat_map['category_id'] = df_cat_map['category_id'].astype(np.int8)

In [8]:
# pull in category name into the base dataframe
df = df.merge(df_cat_map, how='left', on='category_id')
df.shape

(33089, 17)

In [9]:
df.category_title.isnull().sum()

103

In [10]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33089 entries, 0 to 33088
Data columns (total 17 columns):
video_id                  33089 non-null object
trending_date             33089 non-null datetime64[ns]
title                     33089 non-null object
channel_title             33089 non-null object
category_id               33089 non-null int64
publish_time              33089 non-null datetime64[ns, UTC]
tags                      33089 non-null object
views                     33089 non-null int64
likes                     33089 non-null int64
dislikes                  33089 non-null int64
comment_count             33089 non-null int64
thumbnail_link            33089 non-null object
comments_disabled         33089 non-null bool
ratings_disabled          33089 non-null bool
video_error_or_removed    33089 non-null bool
description               32562 non-null object
category_title            32986 non-null object
dtypes: bool(3), datetime64[ns, UTC](1), datetime64[ns](1), int64

### Create new features

In [11]:
# days from publish to trending
df['days_to_trending'] = (df['trending_date'] - pd.to_datetime(df['publish_time'].dt.date)).dt.days

In [17]:
# get unique list of tags
list_unique_tag_combo = df.tags.unique()
individual_tags = set()
print(f'# Unique tag combinations: {len(list_unique_tag_combo)}')
for y in tqdm(list_unique_tag_combo):
    individual_tags = individual_tags | set([x.replace('"','') for x in y.split('|')])
list_individual_tags = list(individual_tags)
print(f'# unique tags:{len(list_individual_tags)}')

# Unique tag combinations: 12578


NameError: name 'tqmd' is not defined

In [28]:
df_tag_unique = df[['tags']].drop_duplicates()

In [31]:
df_tag_unique.shape

(12578, 1)

In [36]:
cv = CountVectorizer()
t = pd.DataFrame(cv.fit_transform(df_tag_unique.tags.fillna('').str.replace(r'\|', ' ')).A,columns=cv.get_feature_names())

In [42]:
(t.sum()>100).sum()

879

In [30]:
df_tag_counts = pd.get_dummies(df_tag_unique.tags.str.split('|',expand=True).replace('"','').stack()).sum(level=0)

MemoryError: Unable to allocate array with shape (241297, 103722) and data type uint8

In [23]:
list_individual_tags_counts = []
for x in tqdm(list_individual_tags):
    list_individual_tags_counts.append(df.tags.str.contains(x, case=False, regex=False).sum())

:19<1:06:40, 24.18it/s]

  5%|▍         | 4676/101406 [03:19<1:06:46, 24.15it/s]

  5%|▍         | 4679/101406 [03:19<1:06:49, 24.12it/s]

  5%|▍         | 4682/101406 [03:19<1:06:33, 24.22it/s]

  5%|▍         | 4685/101406 [03:19<1:06:41, 24.17it/s]

  5%|▍         | 4688/101406 [03:19<1:07:34, 23.85it/s]

  5%|▍         | 4691/101406 [03:19<1:07:04, 24.03it/s]

  5%|▍         | 4694/101406 [03:19<1:06:23, 24.28it/s]

  5%|▍         | 4697/101406 [03:19<1:05:36, 24.57it/s]

  5%|▍         | 4700/101406 [03:20<1:06:10, 24.36it/s]

  5%|▍         | 4703/101406 [03:20<1:06:34, 24.21it/s]

  5%|▍         | 4706/101406 [03:20<1:06:51, 24.11it/s]

  5%|▍         | 4709/101406 [03:20<1:07:22, 23.92it/s]

  5%|▍         | 4712/101406 [03:20<1:07:44, 23.79it/s]

  5%|▍         | 4715/101406 [03:20<1:07:20, 23.93it/s]

  5%|▍         | 4718/101406 [03:20<1:06:54, 24.09it/s]

  5%|▍         | 4721/101406 [03:20<1:06:45, 24.14it/s]

  5%|▍         | 4724/101406 [03:21<1:06:20, 24.29it/s]

  5%|▍

KeyboardInterrupt: 

In [ ]:
len(list_individual_tags_counts)

In [81]:
# process video tags
translator = Translator()

# create a dict to store translations
dict_individual_tags = dict()

# create dictionary for translated text
for x in list_individual_tags:
    try:
        dict_individual_tags[x] = translator.translate(x).text
    except:
        dict_individual_tags[x] = x

In [ ]:
# subset of tags that have been translated 
dict_tags_translated = dict()
for key,value in dict_individual_tags.items():
    if key!=value:
        dict_tags_translated[key] = value
print(f'# Translated tags: {len(dict_tags_translated)}')

# save translated tags for future reference
with open('translated_tags.pickle', 'wb') as handle:
    pickle.dump(dict_tags_translated, handle)

## EDAs

In [40]:
px.histogram(df, x='days_to_trending')